In [ ]:
from modules.lstm_encoder import LSTMEncoder
from modules.lstm_decoder import LSTMDecoder
from keras.regularizers import l2
import os
import numpy as np

## Encoding

In [ ]:
encoder = LSTMEncoder(l2, './features/lstm/2_steps/', './convnet_weights/lstm/base/weights.19-1.13.hdf5')

In [ ]:
features_dev = encoder.encode('dev')

In [ ]:
features_dev.shape

## Classes

In [ ]:
indices = np.load('./features/lstm/2_steps/indices_dev.npy')
true_classes = np.load('./features/res_net/classes_dev_make-model.npy')[indices]
true_classnames = np.load('./features/res_net/classnames_dev_make-model.npy')[indices]
true_filenames = np.load('./features/res_net/filenames_dev.npy')[indices]

In [ ]:
true_classes

In [ ]:
true_classnames

In [ ]:
true_filenames

## Predictions

In [ ]:
def map_predictions_to_classnames(predictions, true_classes, classnames):
    classes, indices = np.unique(true_classes, return_index=True)
    
    class_to_index = {}
    for cl, index in zip(classes, indices):
        class_to_index[cl] = index
    
    predicted_classnames = list(map(lambda cl: classnames[class_to_index[cl]], predictions))
    return np.array(predicted_classnames)

In [ ]:
decoder = LSTMDecoder(None, None, features_dev, true_classes, weights_path='./convnet_weights/lstm/decoder_lstm/weights.36-1.27.hdf5')

In [ ]:
predictions = decoder.predict('dev')

In [ ]:
predicted_classnames = map_predictions_to_classnames(predictions, true_classes, true_classnames)

In [ ]:
predictions[predictions != true_classes].shape